# <center>Advanced MPI</center>

## Blocking and Non-blocking Communication in MPI

### Blocking Communication

Blocking communication operations in MPI, such as `MPI_Send` and `MPI_Recv`, are operations that do not return control to the user program until they are completed. This means that when a process calls `MPI_Send`, it will wait until the data has been copied out of the send buffer and is safe to be modified or reused. Similarly, when `MPI_Recv` is called, the process will wait until the data has been fully received and placed into the receive buffer.

Blocking operations are straightforward to use and are suitable for many applications. However, they can lead to inefficiencies in some scenarios where processes need to wait for each other, resulting in idle time.

![Blocking Communication](./img/blocking-comm.png)

**Example: Blocking Send and Receive**

This example demonstrates a simple blocking send and receive operation where one process sends a message to another.

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    if (world_rank == 0) {
        int data = 100;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent data %d to process 1\n", data);
    } else if (world_rank == 1) {
        int data;
        MPI_Recv(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received data %d from process 0\n", data);
    }

    MPI_Finalize();
    return 0;
}
```

In [ ]:
!srun mpicc code/blocking_send_recv.c -o code/blocking_send_recv.o

In [ ]:
#program using 2,3,4 processes per node and using 2 and 3 nodes:
!srun -N 2 ./code/blocking_send_recv.o

### Non-blocking Communication

Non-blocking communication operations in MPI, such as `MPI_Isend` and `MPI_Irecv`, allow a process to initiate a communication operation and then proceed with other computations or communications without waiting for the communication to complete. These functions return immediately, providing a `MPI_Request` object that can be used to check the status or wait for the operation to complete.

Non-blocking operations are useful for overlapping communication with computation, potentially improving the performance of parallel applications by reducing idle time.
![Non Blocking Communication](./img/non-blocking-comm.png)


**Example: Non-blocking Send and Receive**

This example demonstrates non-blocking send and receive operations where one process sends a message to another, but both processes can perform other work while waiting for the communication to complete.

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    int data;
    MPI_Request request;
    MPI_Status status;

    if (world_rank == 0) {
        data = 123;
        MPI_Isend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
        printf("Process 0 initiated non-blocking send of data %d\n", data);
        // Perform some work while the send operation completes
        printf("Process 0 is doing other work while waiting for send to complete\n");
        MPI_Wait(&request, &status);  // Ensure the send operation is complete
    } else if (world_rank == 1) {
        MPI_Irecv(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &request);
        printf("Process 1 initiated non-blocking receive\n");
        // Perform some work while the receive operation completes
        printf("Process 1 is doing other work while waiting for receive to complete\n");
        MPI_Wait(&request, &status);  // Ensure the receive operation is complete
        printf("Process 1 received data %d\n", data);
    }

    MPI_Finalize();
    return 0;
}```

In [ ]:
!srun mpicc code/unit_1.c -o code/unit_1.o

In [ ]:
# Exercise No 4
# Save the program as `non_blocking_send_recv.c` on code folder code and compile it using `mpicc`:

!mpicc ...

#program using 2,3,4 processes per node and using 2 and 3 nodes:

!mpirun ...


### Comparison of Blocking and Non-blocking Communication

1. **Blocking Communication**:
   - **Pros**: Simplicity, straightforward usage.
   - **Cons**: Potential for idle time, processes may wait for each other.

2. **Non-blocking Communication**:
   - **Pros**: Overlap communication with computation, potential performance improvements.
   - **Cons**: More complex to use, requires careful management of `MPI_Request` objects and completion checks.

Using non-blocking communication effectively requires a good understanding of the application’s communication and computation patterns, enabling the overlap of these operations to maximize performance.

In [ ]:
!mpirun -np 4 ./hello_world # This example uses 4 processes on the same machine 
!!srun -N 2 -n 4 --ntasks-per-node=2 ./hello_world  # This example uses 4 processes on 2 nodes

Note that the execution block is enclosed in a function called `main()`, which returns the value 0 if it is completed successfully. The declaration of `main()` is mandatory in C/C++.

In [ ]:
!mpicc code/unit_1.c -o code/unit_1

In [ ]:
!srun -N 2 -n 4 --ntasks-per-node=2 ./code/unit_1


## Unit 2: MPI Collective Communication

The Type of Collective communication on MPI are:

![Collective](./img/collective_comm.gif)

### Subtopic 2.1: Broadcast
#### Explanation
- **MPI_Bcast**: Broadcasts a message from the process with rank "root" to all other processes in the communicator.

#### Example: Broadcast
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    int data = 0;
    if (world_rank == 0) {
        data = 100;
    }
    MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);
    printf("Process %d received data %d\n", world_rank, data);

    MPI_Finalize();
    return 0;
}
```

### MPI  Data Types

To share data between nodes, is required use same data types, to cast values and manipulate side to side. MPI predefines its primitive data types:


|C Data Types 1| C Data Types 2|
|   ---  |  --- |	
|MPI_CHAR<br/>MPI_WCHAR<br/>MPI_SHORT<br/>MPI_INT<br/>MPI_LONG<br/>MPI_LONG_LONG_INT<br/>MPI_LONG_LONG<br/>MPI_SIGNED_CHAR<br/>MPI_UNSIGNED_CHAR<br/>MPI_UNSIGNED_SHORT<br/>MPI_UNSIGNED_LONG<br/>MPI_UNSIGNED<br/>MPI_FLOAT<br/>MPI_DOUBLE<br/>MPI_LONG_DOUBLE|MPI_C_COMPLEX<br/>MPI_C_FLOAT_COMPLEX<br/>MPI_C_DOUBLE_COMPLEX<br/>MPI_C_LONG_DOUBLE_COMPLEX<br/>MPI_C_BOOL<br/>MPI_LOGICAL<br/>MPI_C_LONG_DOUBLE_COMPLEX<br/>MPI_INT8_T<br/>MPI_INT16_T<br/>MPI_INT32_T<br/>MPI_INT64_T<br/>MPI_UINT8_T<br/>MPI_UINT16_T<br/>MPI_UINT32_T<br/>MPI_UINT64_T<br/>MPI_BYTE<br/>MPI_PACKED|



|MPI Reduction Operation|	C Data Types	|
|   --- |   --- |
|MPI_MAX|	maximum	|integer, float	|
|MPI_MIN|	minimum	integer, float	|
|MPI_SUM|	sum	|integer, float	|
|MPI_PROD|	product	|integer, float	|
|MPI_LAND|	logical AND|	integer	|
|MPI_BAND|	bit-wise AND|integer MPI_BYTE|	
|MPI_LOR|	logical OR|	integer	|
|MPI_BOR|	bit-wise OR	integer, MPI_BYTE|	
|MPI_LXOR|	logical XOR	|integer	
|MPI_BXOR|	bit-wise XOR	|integer, MPI_BYTE|
|MPI_MAXLOC|	max value and location	|float, double and long double|	
|MPI_MINLOC|	min value and location	|float, double and long double|	


### Exercise No 2. 

Please compile  with support for MPI and Submit it to Job Manager to compile and run the example of `distributed_sum.c`, use several configuration to runtime: Example 2 nodes with 2 Process per node,  with 4 process per node, 3, 4 Nodes too.  Please comment the results. 

In [ ]:
!srun mpicc exercises/unit_1.c -o exercises/unit_1

In [ ]:
!srun -N 2 -n 4 --ntasks-per-node=2 ./exercises/unit_1